In [7]:
import numpy as np

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

from sklearn.utils import resample

from warnings import filterwarnings
filterwarnings('ignore')


train_df = pd.read_csv('./data/mitbih_train.csv', header=None)
test_df = pd.read_csv('./data/mitbih_test.csv', header=None)
weight_path = "weights_lstm.hdf5"

MITBIH_raw = pd.concat([train_df, test_df], axis=0).sample(frac=1, axis=0)

test_df_MI  = MITBIH_raw[0:len(MITBIH_raw)//10]
train_df = MITBIH_raw[len(MITBIH_raw)//10:]

df_1 = train_df[train_df[187] == 1]
df_2 = train_df[train_df[187] == 2]
df_3 = train_df[train_df[187] == 3]
df_4 = train_df[train_df[187] == 4]
df_0 = (train_df[train_df[187] == 0]).sample(n=20000, random_state=42)

df_1_upsample = resample(df_1, replace=True, n_samples=20000, random_state=123)
df_2_upsample = resample(df_2, replace=True, n_samples=20000, random_state=124)
df_3_upsample = resample(df_3, replace=True, n_samples=20000, random_state=125)
df_4_upsample = resample(df_4, replace=True, n_samples=20000, random_state=126)

train_df = pd.concat([df_0, df_1_upsample, df_2_upsample, df_3_upsample, df_4_upsample])

y_train = to_categorical(train_df[187], 5)
y_test = to_categorical(test_df[187], 5)

x_train = train_df.iloc[:, :187].values
x_test = test_df.iloc[:, :187].values
x_train = x_train.reshape(len(x_train), x_train.shape[1], 1)
x_test = x_test.reshape(len(x_test), x_test.shape[1], 1)

train_weights = True
batch_size = 64
epochs = 20

model = Sequential()
model.add(LSTM(64, input_shape=(187, 1)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax'))
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

checkpoint = ModelCheckpoint(weight_path, monitor='loss', verbose=1, save_best_only=True, mode='min')

if train_weights:
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              callbacks=[checkpoint])

else:
    model.load_weights(weight_path)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print(score, acc)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645       
Total params: 25,861
Trainable params: 25,861
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1563/1563 [==============================] - 108s 68ms/step - loss: 1.2983 - accuracy: 0.4390 - val_loss: 1.0631 - val_accuracy: 0.6199

Epoch 00001: loss improved from inf to 1.11881, saving model to weights_lstm.hdf5
Epoch 2/20
1563/15

KeyboardInterrupt: 